In [ ]:
# create hgnc_symbols.csv

import json
import csv

symbols = []

with open('hgnc_all.json', 'r') as f:
    data = json.load(f)  # Load the entire file as a list

for obj in data:
    symbols.append(obj['approvedSymbol'])

with open('hgnc_symbols_all.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['approvedSymbol'])  # header
    for symbol in symbols:
        writer.writerow([symbol])

In [ ]:
# filter out olfaction + taste hgnc symbols

import pandas as pd

# Read the original CSV
df = pd.read_csv('hgnc_symbols.csv')

# Filter out rows where 'approvedSymbol' starts with 'OR' or 'TAS'
df = df[~df['approvedSymbol'].str.startswith(('OR', 'TAS'))]

# Overwrite the original CSV with the filtered data
df.to_csv('hgnc_symbols.csv', index=False)

In [ ]:
# write human orthos txt to csv file

import pandas as pd

rows = []
with open('human_orthos_2025.05.16.txt') as f:
    for i, line in enumerate(f, 1):
        if i < 3:
            continue  # skip the first line
        fields = line.rstrip('\n').split('\t')
        if len(fields) == 13:
            rows.append(fields) 

columns = [
    'ZFIN ID', 'ZFIN Symbol', 'ZFIN Name', 'Human Symbol', 'Human Name', 'OMIM ID', 'Gene ID', 'HGNC ID',
    'Evidence', 'Pub ID', 'ZFIN Abbreviation Name', 'ECO ID', 'ECO Term Name'
]

df = pd.DataFrame(rows, columns=columns)
df.to_csv('human_orthos.csv', index=False)

In [ ]:
# create list of ALL human matches (including duplicates)

import pandas as pd

# Read the list of symbols
hgnc_df = pd.read_csv('hgnc_symbols.csv')
symbols = hgnc_df['approvedSymbol'].tolist()

# Read the human orthologs table
orthos_df = pd.read_csv('human_orthos.csv')

# Filter rows where 'Human Symbol' matches any symbol in the list
filtered = orthos_df[orthos_df['Human Symbol'].isin(symbols)]

# Select only the desired columns
result = filtered[['Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID']]

# Save to CSV
result.to_csv('matched_human_orthos.csv', index=False)

In [28]:
# sort match_human_orthos.csv into match_human_orthos_unique.csv and match_human_orthos_dupes.csv

import pandas as pd

# Read the matched human orthos file
df = pd.read_csv('matched_human_orthos.csv')

# Count occurrences of each ZFIN Symbol
counts = df['ZFIN Symbol'].value_counts()

# Unique: ZFIN Symbol appears only once
unique = df[df['ZFIN Symbol'].isin(counts[counts == 1].index)]

# Dupes: ZFIN Symbol appears more than once, keep only the first occurrence
dupes = df[df['ZFIN Symbol'].isin(counts[counts > 1].index)].drop_duplicates(subset=['ZFIN Symbol'])
dupes['Count'] = dupes['ZFIN Symbol'].map(counts)

# Save to CSV
unique.to_csv('matched_human_orthos_unique.csv', index=False)
dupes.to_csv('matched_human_orthos_dupes.csv', index=False)

In [ ]:
# create unmatched_human_orthos.csv, which contains anything that was not matched (sorted by unique and dupes)

import pandas as pd

import pandas as pd

# Headers for two rows (12 columns)
header1 = ['Unique', '', '', '', '', '', '', 'Dupes', '', '', '', '']
header2 = [
    'Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID', '', '', '',
    'Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID', 'Count (zf)'
]

# Read data
hgnc_df = pd.read_csv('hgnc_symbols.csv')
symbols = set(hgnc_df['approvedSymbol'].tolist())
orthos_df = pd.read_csv('human_orthos.csv')

# Find unmatched orthologs
unmatched = orthos_df[~orthos_df['Human Symbol'].isin(symbols)]

# Print any ZFIN Symbols that are duplicated in unmatched
dupe_symbols = unmatched['ZFIN Symbol'][unmatched['ZFIN Symbol'].duplicated(keep=False)]
if not dupe_symbols.empty:
    print("Duplicate ZFIN Symbols in unmatched:")
    print(dupe_symbols.value_counts())
else:
    print("No duplicate ZFIN Symbols in unmatched.")

# Count occurrences of each ZFIN Symbol in unmatched
counts = unmatched['ZFIN Symbol'].value_counts()

# Unique: ZFIN Symbol appears only once
unique = unmatched[unmatched['ZFIN Symbol'].isin(counts[counts == 1].index)][['Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID']]

# Dupes: ZFIN Symbol appears more than once, keep only the first occurrence, add Count
dupes = unmatched[unmatched['ZFIN Symbol'].isin(counts[counts > 1].index)].drop_duplicates(subset=['ZFIN Symbol']).copy()
dupes['Count (zf)'] = dupes['ZFIN Symbol'].map(counts)
dupes = dupes[['Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID', 'Count (zf)']]

# Make both DataFrames the same length for side-by-side display
max_len = max(len(unique), len(dupes))
unique_padded = unique.reindex(range(max_len)).reindex(columns=['Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID', '', '', ''])
dupes_padded = dupes.reindex(range(max_len)).reindex(columns=['Human Symbol', 'ZFIN ID', 'ZFIN Symbol', 'Pub ID', 'Count (zf)'])

# Concatenate side by side (7 + 5 = 12 columns)
final = pd.concat([unique_padded, dupes_padded], axis=1)

# Set MultiIndex columns for two header rows
final.columns = pd.MultiIndex.from_arrays([header1, header2])

# Drop rows where all columns are empty or NaN
final = final.dropna(how='all')
final = final[~final.apply(lambda row: all(str(x).strip() == '' or pd.isna(x) for x in row), axis=1)]

# Save to CSV
final.to_csv('unmatched_human_orthos.csv', index=False)


Duplicate ZFIN Symbols in unmatched:
ZFIN Symbol
slc2a1a      18
zc4h2        16
irf4a        15
slc2a1b      15
rarab        14
             ..
ccdc59        2
glt8d2        2
tspan5b       2
aimp1b        2
snord118a     2
Name: count, Length: 14758, dtype: int64
Number of unmatched human orthologs: 16624


In [35]:
# print number of non-empty rows (value added manually to unmatched_human_orthos.csv)

import csv

count = 0
with open('unmatched_human_orthos.csv', newline='', encoding='utf-8') as f:
    reader = csv.reader(f)
    for i, row in enumerate(reader, 1):
        if i < 3:
            continue  # skip the first two header rows
        if any(cell.strip() for cell in row):
            count += 1

print(f"Number of non-empty rows (starting from line 3): {count}")

Number of non-empty rows (starting from line 3): 5328
